# Load

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pytesseract
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image
import cv2
from google.colab.patches import cv2_imshow
import math
import numpy as np
import cv2
import pandas as pd

Class for opening videos and extracting frames:

In [ ]:
class Video:
  def __init__(self, path):
    self.path = path
    self.video = cv2.VideoCapture(self.path)
    self.cur_frame = 0
    self.fps = self.video.get(cv2.CAP_PROP_FPS)

  def get_frame_id(self, minutes, seconds):
    frame_id = int(self.fps*(minutes*60+seconds))
    return frame_id

  def set_frame(self, frame_id):
    self.video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
    self.cur_frame = frame_id

  def get_frame(self, frame_id):
    self.set_frame(frame_id)
    success, frame = self.video.read()
    return success, frame

  def get_cur_frame(self):
    success, frame = self.video.read()
    return success, frame

  def get_set_frame(self, minutes, seconds):
    self.set_frame_time(minutes, seconds)
    return self.get_cur_frame()

  def get_timeframe(self, start_minutes, start_seconds, end_minutes, end_seconds, frame_rate):
    n_frames = (self.get_frame_id(end_minutes, end_seconds) - self.get_frame_id(start_minutes, start_seconds)) // frame_rate
    cur_frame_id = self.get_frame_id(start_minutes, start_seconds)
    frame_list = []
    for i in range(0, n_frames):
      succes, frame = self.get_frame(cur_frame_id)
      frame_list.append(frame)
      cur_frame_id += frame_rate
    return frame_list

# Extract Scores

In [ ]:
import cv2
import pytesseract
import pandas as pd

songs = ["americano", "diggin", "jagger", "pink"]
participants = ["part1", "part2", "part3", "part4", "part5", "part6"]

# Set the frame interval
frame_interval = 1

scores = ["X", "PERFECT", "GOOD", "OK", "YEAH"]

for song in songs:

  # Load the dataframe that consists of the time intervals of the scores
  df = pd.read_csv ('/content/drive/MyDrive/Thesis/Experiment Scores/{}.csv'.format(song))

  for part in participants:

    cur_frame_id = 0
    score_list = []

    print('processing: {} - {}...'.format(song, part))
    vid = Video('/content/drive/MyDrive/Thesis/Experiment Scores/{}_{}.mp4'.format(song, part))

    for index, row in df.iterrows():

      start_time = row['start_time']
      start_minute, start_second = start_time.split(':')

      end_time = row['end_time']
      end_minute, end_second = end_time.split(':')

      frame_list = vid.get_timeframe(int(start_minute), float(start_second), int(end_minute), float(end_second), 1)

      score_dic = {"X":0, "PERFECT":0, "GOOD":0, "OK":0, "YEAH":0}

      # Loop through the frames
      for frame in frame_list:

          # Convert the image to grayscale
          gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          grey_inv = cv2.bitwise_not(gray)

          # Recognize the string using OCR
          text = pytesseract.image_to_string(grey_inv, config='--psm 6')

          # Print the recognized text
          for score in scores:
            if score in text.upper():
                score_dic[score] += 1

      score_dic = sorted(score_dic.items(), key=lambda x:x[1], reverse=True)
      score_list.append(score_dic[0][0])
      #print(index, ": ", score_dic[0][0])


    pred_score_df = df
    pred_score_df['score'] = score_list

    pred_score_df.to_csv('/content/drive/MyDrive/Thesis/Experiment Scores/{}_{}_pred.csv'.format(song, part))
    print('{} - {} done!'.format(song, part))

processing: pink - part5...
pink - part5 done!
processing: pink - part6...
pink - part6 done!
processing: diggin - part5...
diggin - part5 done!
processing: diggin - part6...
diggin - part6 done!


# Test Accuracy

In [ ]:
import sklearn
from sklearn.metrics import accuracy_score

true_files = ['jagger_part4', 'pink_part3', 'diggin_part1', 'diggin_part2', 'americano_part1', 'americano_part3']
all_acc = []

for true in true_files:

  true_scores = pd.read_csv ('/content/drive/MyDrive/Thesis/Experiment Scores/{}_true.csv'.format(true))
  y_true = true_scores['score']

  pred_scores = pd.read_csv ('/content/drive/MyDrive/Thesis/Experiment Scores/{}_pred.csv'.format(true))
  y_pred = pred_scores['score']

  acc = sklearn.metrics.accuracy_score(y_true, y_pred, normalize=True)
  print(true, ':', acc)
  all_acc.append(acc)

av_acc = sum(all_acc) / len(all_acc)
print('average: ', av_acc)